### Unit 2: Lesson 4: Drill: Multivariable Regression: New York Crime Data
Saheed Tijani

Dec 2018

In [176]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [177]:
#grab the raw data

data_raw = pd.read_csv('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013_csv.csv', header=None)

#### Data cleaning

In [178]:
#remove data rows(0,1,2 and 3), and the last 3 rows. This row are only describe the data. They do not have statistical values.
#drop the rape1 col as well as it has not observation.
data_cleaning = data_raw.drop(data_raw.index[:5]).drop(data_raw.index[-3:])
#.drop('rape_1', axis=1)

#name data columns
data_cleaning.columns = ['city', 'population', 'violent_crime', 'murder_manslaughter',
       'rape_1', 'rape_2', 'robbery', 'aggravated_assault', 'property_crime', 
       'burglary', 'larceny_theft', 'mv_theft', 'arson_3']

#drop the rape1 col as it has no observation.
data_cleaning = data_cleaning.drop('rape_1', axis=1)

# Remove commas in numbers in relevant cols
data_cleaning['population'] = data_cleaning['population'].str.replace(',', '')
data_cleaning['robbery'] = data_cleaning['robbery'].str.replace(',', '')

#data is now clean and ready for feature engineering, let's name the clean data
data_cleaned = data_cleaning

#overview overview of clean data
data_cleaned.head()

,city,population,violent_crime,murder_manslaughter,rape_2,robbery,aggravated_assault,property_crime,burglary,larceny_theft,mv_theft,arson_3
5,Adams Village,1861,0,0,0,0,0,12,2,10,0,0
6,Addison Town and Village,2577,3,0,0,0,3,24,3,20,1,0
7,Akron Village,2846,3,0,0,0,3,16,1,15,0,0
8,Albany,97956,791,8,30,227,526,"4,090",705,"3,243",142,NaN
9,Albion Village,6388,23,0,3,4,16,223,53,165,5,NaN


### Creating Features

In [179]:
#create the population-square feature
data_cleaned['population_square'] =  (data_cleaned['population']).astype(int)**2

#create a function to categrize continous data
def cont_2_cat(col):
    for element in col:
        if int(element) > 1:
            return 1
        else:
            return 0
#apply our cont_2_cat function to catgorize data_cleaned.murder_manslaughter and create feature with the result       
data_cleaned['murder_manslaughter_categorical'] = data_cleaned.murder_manslaughter.apply(cont_2_cat)

#use the npwhere method to categorize roberry and create a feature with it
#I could'nt figure out why - feature_data['robbery_categorical'] = np.where(int(data_cleaned['robbery']) > 1 ,1,0) did not work
data_cleaned['robbery_categorical'] = np.where((data_cleaned['robbery']).astype(int) > 1 ,1,0)

#final data overview
data_cleaned

,city,population,violent_crime,murder_manslaughter,rape_2,robbery,aggravated_assault,property_crime,burglary,larceny_theft,mv_theft,arson_3,population_square,murder_manslaughter_categorical,robbery_categorical
5,Adams Village,1861,0,0,0,0,0,12,2,10,0,0,3463321,0,0
6,Addison Town and Village,2577,3,0,0,0,3,24,3,20,1,0,6640929,0,0
7,Akron Village,2846,3,0,0,0,3,16,1,15,0,0,8099716,0,0
8,Albany,97956,791,8,30,227,526,"4,090",705,"3,243",142,NaN,1005443344,1,1
9,Albion Village,6388,23,0,3,4,16,223,53,165,5,NaN,40806544,0,1
10,Alfred Village,4089,5,0,0,3,2,46,10,36,0,NaN,16719921,0,1
11,Allegany Village,1781,3,0,0,0,3,10,0,10,0,0,3171961,0,0
12,Amherst Town,118296,107,1,7,31,68,"2,118",204,"1,882",32,3,1109041728,0,1
13,Amityville Village,9519,9,0,2,4,3,210,16,188,6,1,90611361,0,1
14,Amsterdam,18182,30,0,0,12,18,405,99,291,15,0,330585124,0,1
